<a href="https://colab.research.google.com/github/utkarsh599/Recomendation-sysy/blob/Abhijeet/learningSpark6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
!pip install py4j

     |████████████████████████████████| 204kB 3.1MB/s 


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()

In [0]:
import pyspark

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
from pyspark.sql import SparkSession


In [0]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
comments1 =  spark.read.csv("/content/gdrive/My Drive/Colab Notebooks/GBcomments.csv", inferSchema=True, header=True)

In [0]:
comments2= spark.read.csv("/content/gdrive/My Drive/Colab Notebooks/UScomments.csv", inferSchema=True, header=True)

In [0]:
comments1.show(5)

In [0]:
type(comments1)

In [0]:
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

comments=unionAll(comments1,comments2)

In [0]:
comments.count()

In [0]:
comments_rdd = comments.select('comment_text').rdd.flatMap(lambda x: x)
comments_rdd=comments_rdd.filter(lambda x: x is not None).filter(lambda x: x!='')
comments_rdd.take(5)

In [0]:
header = comments_rdd.first()

In [0]:
data_rmv_col = comments_rdd.filter(lambda row: row != header)

In [0]:
def lower(x):
  return x.lower()

In [0]:
lowerCase_sentRDD = data_rmv_col.map(lower)

In [0]:
lowerCase_sentRDD.take(5)

In [0]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [0]:
def sent_TokenizeFunct(x):
    doc=nlp(x)
    #return doc
    tokens=[]
    for token in doc:
      if((token.is_stop==False and token.is_punct==False) and token.is_alpha==True):
        tokens.append(token.lemma_)
    return ' '.join(tokens)
    
sentenceTokenizeRDD = lowerCase_sentRDD.map(sent_TokenizeFunct)

In [0]:
sentenceTokenizeRDD.take(5)

# Working on Videos

In [0]:
Videos1 =  spark.read.csv("/content/gdrive/My Drive/Colab Notebooks/GBvideos.csv", inferSchema=True, header=True)
Videos2 =  spark.read.csv("/content/gdrive/My Drive/Colab Notebooks/USvideos.csv", inferSchema=True, header=True)
videos = unionAll(Videos1,Videos2)

In [0]:
videos.show(5)

In [0]:
videos_rdd = videos.select(['title']).rdd.flatMap(lambda x: x)
videos_rdd = videos_rdd.filter(lambda x: x is not None).filter(lambda x: x!='')
videos_rdd.take(5)

In [0]:
header_Vid=videos_rdd.first()

In [0]:
data_rmv_col_vid = videos_rdd.filter(lambda row: row != header_Vid)

In [0]:
lowerCase_VidRDD = data_rmv_col_vid.map(lower)

In [0]:
lowerCase_VidRDD.take(5)

In [0]:
Vid_sentTokenizeRDD = lowerCase_VidRDD.map(sent_TokenizeFunct)

In [0]:
Vid_sentTokenizeRDD.take(5)

In [0]:
tags_rdd=videos.select(['tags']).rdd.flatMap(lambda x:x)
tags_rdd.take(5)

In [0]:
header_tags = tags_rdd.first()

In [0]:
tags_rmv_col = tags_rdd.filter(lambda row: row != header_tags)

In [0]:
tags_rmv_col.take(5)

In [0]:
def split1(x):
  return (x.split("|"))

tags_rdd=tags_rmv_col.map(split1)

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import monotonically_increasing_id

row_title = Row("LemmetizeTitle") # Or some other column name
Title_DF = Vid_sentTokenizeRDD.map(row_title).toDF()
Title_DF = Title_DF.select("*").withColumn("id", monotonically_increasing_id())

row_tags=Row("Tokenized_Tags")
Tags_DF=tags_rdd.map(row_tags).toDF()
Tags_DF = Tags_DF.select("*").withColumn("id", monotonically_increasing_id())

In [0]:
Title_DF.show(5)

In [0]:
Tags_DF.show(5)

In [0]:
Tags_DF[['Tokenized_Tags']]

In [0]:
videos_DF = Title_DF.join(Tags_DF, on=['id'], how='left_outer')

In [0]:
videos_DF.show(5)

In [0]:
videos_DF.count()

In [0]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer


tokenizer = Tokenizer(inputCol="LemmetizeTitle", outputCol="words")
wordsData = tokenizer.transform(videos_DF)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=5)
featurizedData = hashingTF.transform(wordsData)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

rescaledData.select("id", "features").show()

In [0]:
rescaledData.select("id", "features").show(n=5, truncate=False, vertical=True)

In [0]:
trainSDF,validationSDF,TestSDF=rescaledData.select("id","Tokenized_tags").randomSplit([0.75, 0.2, 0.05])

In [0]:
trainSDF.show(5)

In [0]:
validationSDF.show(5)

In [0]:
TestSDF.count()

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, MinHashLSH
import pyspark.sql.functions as f

model = Pipeline(stages=[
        HashingTF(inputCol="Tokenized_tags", outputCol="vectors"),
        MinHashLSH(inputCol="vectors", outputCol="lsh", numHashTables=10)
    ]).fit(TestSDF)

db_hashed = model.transform(TestSDF)

db_matches = model.stages[-1].approxSimilarityJoin(db_hashed, db_hashed, 0.9)

#show all matches (including duplicates)
db_matches.select(f.col('datasetA.id').alias('video_id_A'),
                 f.col('datasetB.id').alias('video_id_B'),
                 f.col('distCol')).show()


In [0]:
from pyspark.sql.functions import col
#show non-duplicate matches
ans=db_matches.select(f.col('datasetA.id').alias('video_id_A'),
                 f.col('datasetB.id').alias('video_id_B'),
                 f.col('distCol')).filter('video_id_A < video_id_B')

ans=ans.sort(col("distCol").desc())
ans.show()

In [0]:
set1,set2,set3=ans.select("video_id_A","video_id_B","distCol").randomSplit([0.33, 0.33, 0.34])

In [0]:
set1.show()

In [0]:
set2.show()

In [0]:
set3.show()

In [0]:
ans.select('*').where(col('video_id_A')="776")

SyntaxError: ignored

In [0]:
# from pyspark.ml.feature import HashingTF, IDF
# hashingTF = HashingTF(inputCol="Tokenized_Tags", outputCol="tf")
# tf = hashingTF.transform(TestSDF.select("id","Tokenized_Tags"))

# idf = IDF(inputCol="tf", outputCol="feature").fit(tf)
# tfidf = idf.transform(tf)

In [0]:
# #   tf.cache()
# #   idf = IDF().fit(tf)
# #   tfidf = idf.transform(tf)

# from pyspark.ml.feature import Normalizer
# normalizer = Normalizer(inputCol="feature", outputCol="norm")
# data = normalizer.transform(tfidf)

In [0]:
# data.show(5)

+----------+--------------------+--------------------+--------------------+--------------------+
|        id|      Tokenized_Tags|                  tf|             feature|                norm|
+----------+--------------------+--------------------+--------------------+--------------------+
|      3091|[affordable makeu...|(262144,[5262,199...|(262144,[5262,199...|(262144,[5262,199...|
|      4894|[Linkin Park, One...|(262144,[21656,84...|(262144,[21656,84...|(262144,[21656,84...|
|      7279|[carlibel55, carl...|(262144,[10116,37...|(262144,[10116,37...|(262144,[10116,37...|
|8589938024|[Full Frontal wit...|(262144,[27323,66...|(262144,[27323,66...|(262144,[27323,66...|
|8589942265|        [nerdwriter]|(262144,[109605],...|(262144,[109605],...|(262144,[109605],...|
+----------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [0]:

# from pyspark.sql.types import DoubleType

In [0]:
# import pyspark.sql.functions as psf
# from pyspark.sql.functions import col
# dot_udf = psf.udf(lambda x,y: float(x.dot(y)), DoubleType())
# result=data.alias("i").join(data.alias("j"), psf.col("i.ID") < psf.col("j.ID"))
# result=result.select(
#         psf.col("i.ID").alias("i"), 
#         psf.col("j.ID").alias("j"), 
#         dot_udf("i.norm", "j.norm").alias("dot"))
    
# # result=result.sort("i", "j")

# result=result.sort(col("dot").desc())
# result.show()

+----------+----------+------------------+
|         i|         j|               dot|
+----------+----------+------------------+
|      2727|8589936472|1.0000000000000007|
|      5060|      5738|1.0000000000000007|
|      7260|      7741|1.0000000000000004|
|      1652|      2366|1.0000000000000004|
|       610|      1262|1.0000000000000002|
|      2759|8589936482|1.0000000000000002|
|      6152|8589939207|1.0000000000000002|
|      4116|8589938314|1.0000000000000002|
|8589938210|8589938672|1.0000000000000002|
|      4135|      4383|1.0000000000000002|
|      5459|8589939207|1.0000000000000002|
|      4417|      5312|1.0000000000000002|
|      7402|8589942486|1.0000000000000002|
|      6015|      7097|1.0000000000000002|
|8589940276|8589940529|1.0000000000000002|
|8589936018|8589936775|1.0000000000000002|
|      5750|      6418|1.0000000000000002|
|      5459|      6152|1.0000000000000002|
|      5213|      6172|1.0000000000000002|
|      7402|8589941791|1.0000000000000002|
+----------

In [0]:

# result.sort(col("dot").desc())

In [0]:
# content based recommender system
# steps: 
# 1) lower case whole dataset
# 2) remove stopwords
# 3) create bag of words vectors in binary format 1/0
# 4) calc cosine sim for all articles and find top 3 closest articles
# 5) sort article ids with descen simi and removing same article id
# 6) created recommendation column with top 3 article id's